<a href="https://colab.research.google.com/github/lokeshparab/GenAI-Full-Course/blob/main/RAG/MM_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and Import Library

In [ ]:
!sudo apt-get update
!sudo apt-get install poppler-utils
!sudo apt-get install tesseract-ocr
!sudo apt-get install libtesseract-dev
!pip install "unstructured[all-docs]" pillow pydantic lxml matplotlib unstructured-pytesseract
!pip install chromadb grandalf langchain langchain-community langchain-groq langchain-pinecone pinecone-notebooks langchain-anthropic langchain-google-genai langchain-openai langchain-huggingface

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [ ]:
from langchain_groq import ChatGroq
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint, HuggingFacePipeline, HuggingFaceEmbeddings

import os

from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
os.environ["ANTHROPIC_API_KEY"] = userdata.get('ANTHROPIC_API_KEY')
os.environ["PINECONE_API_KEY"] = userdata.get('PINECONE_API_KEY')

# MM Extraction

## Method 1

In [ ]:
from unstructured.partition.pdf import partition_pdf

In [ ]:
import os
import requests

def download_pdf(url, save_dir):
    # Ensure the save directory exists
    os.makedirs(save_dir, exist_ok=True)

    # Extract the filename from the URL
    filename = url.split("/")[-1]

    # Construct the full path to save the file
    file_path = os.path.join(save_dir, filename)

    try:
        # Send a GET request to the URL
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Raise an error for bad status codes

        # Write the content to the file in chunks
        with open(file_path, "wb") as f:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)

        print(f"✅ File downloaded successfully: {file_path}")

        return file_path
    except requests.exceptions.RequestException as e:
        print(f"❌ Failed to download file: {e}")


# Example usage
pdf_url = "https://raw.githubusercontent.com/lokeshparab/GenAI-Full-Course/refs/heads/main/data/RAG_FOR_NLP.pdf"
destination_folder = "data"  # Replace with your desired directory path

filename = download_pdf(pdf_url, destination_folder)


✅ File downloaded successfully: data/RAG_FOR_NLP.pdf


In [ ]:
raw_pdf_elements = partition_pdf(
    filename=filename,
    strategy="hi_res",
    extract_images_in_pdf=True,
    extract_image_block_types=["Image","Table"],
    extract_image_block_to_payload=False,
    extract_image_block_output_dir="extracted_data",
    infer_table_structure=True,
)

In [ ]:
Header=[]
Footer=[]
Title=[]
NarrativeText=[]
Text=[]
ListItem=[]
Image=[]
Table=[]
for element in raw_pdf_elements:
  if "unstructured.documents.elements.Header" in str(type(element)):
            Header.append(str(element))
  elif "unstructured.documents.elements.Footer" in str(type(element)):
            Footer.append(str(element))
  elif "unstructured.documents.elements.Title" in str(type(element)):
            Title.append(str(element))
  elif "unstructured.documents.elements.NarrativeText" in str(type(element)):
            NarrativeText.append(str(element))
  elif "unstructured.documents.elements.Text" in str(type(element)):
            Text.append(str(element))
  elif "unstructured.documents.elements.ListItem" in str(type(element)):
            ListItem.append(str(element))
  elif "unstructured.documents.elements.Image" in str(type(element)):
            Image.append(str(element))
  elif "unstructured.documents.elements.Table" in str(type(element)):
            Table.append(str(element))

In [ ]:
Header

['2 r p A 2 1 ] L C . s c [ 4 v 1 0 4 1 1 . 5 0 0 2 :', '16', '19']

In [ ]:
Title

['v',
 'arXiv',
 'i',
 'X',
 'r',
 'a',
 'Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks',
 'Abstract',
 '1 Introduction',
 '2 Methods',
 '2.1 Models',
 'N',
 '2.2 Retriever: DPR',
 'py(z|x) x exp(d(z)"q(x))',
 '2.3 Generator: BART',
 '2.4 Training',
 '2.5 Decoding',
 '3 Experiments',
 '3.1 Open-domain Question Answering',
 '3.2 Abstractive Question Answering',
 'Jeopardy Question Generation',
 '3.4 Fact Veriﬁcation',
 '4 Results',
 '4.1 Open-domain Question Answering',
 'Model',
 'NQ',
 'TQA WQ CT',
 'Jeopardy MSMARCO FVR3 FVR2',
 '4.2 Abstractive Question Answering',
 'Jeopardy Question Generation',
 '4.4 Fact Veriﬁcation',
 'Doc 1 poe 2',
 'Doc 3',
 '4.5 Additional Results',
 'Table 5: Ratio of distinct to total tri-grams for',
 'generation tasks.',
 'FVR-2',
 'Label Accuracy',
 '5 Related Work',
 '6 Discussion',
 'Broader Impact',
 'Acknowledgments',
 'References',
 'Appendices for Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks',
 'A Implementati

In [ ]:
NarrativeText

["Patrick Lewis't, Ethan Perez*,",
 'Aleksandra Piktus†, Fabio Petroni†, Vladimir Karpukhin†, Naman Goyal†, Heinrich Küttler†,',
 'Mike Lewis†, Wen-tau Yih†, Tim Rocktäschel†‡, Sebastian Riedel†‡, Douwe Kiela†',
 '+Racebook AI Research; *University College London; *New York University;',
 'plewis@fb.com',
 'Large pre-trained language models have been shown to store factual knowledge in their parameters, and achieve state-of-the-art results when ﬁne-tuned on down- stream NLP tasks. However, their ability to access and precisely manipulate knowl- edge is still limited, and hence on knowledge-intensive tasks, their performance lags behind task-speciﬁc architectures. Additionally, providing provenance for their decisions and updating their world knowledge remain open research problems. Pre- trained models with a differentiable access mechanism to explicit non-parametric memory have so far been only investigated for extractive downstream tasks. We explore a general-purpose ﬁne-tuning recipe

In [ ]:
Text

['1', '2021', '2', '0', '92.2*', '6', '9']

In [ ]:
Table

['Closed Book T5-11B [52] 34.5 T5-11B+SSM[52] 36.6 - - /50.1 37.4 /60.5 44.7 - - Model B-1 QB-1 R-L B-1 Label Acc. Open REALM [20] 40.4 - / - 40.7 46.8 SotA - - 49.8* 49.9* 76.8 Book DPR [26] 41.5 57.9/ - 41.1 50.6 BART 15.1 19.7 38.2 41.6 64.0 81.1 RAG-Token RAG-Seq. 44.1 55.2/66.1 45.5 50.0 44.5 56.8/68.0 45.2 52.2 RAG-Tok. 17.3 22.2 RAG-Seq. 14.7 21.4 40.1 40.8 41.5 44.2 72.5 89.5',
 'Task Input Model Generation MS- deﬁne middle ear BART ?The middle ear is the part of the ear between the middle ear and the nose. RAG-T The middle ear is the portion of the ear internal to the eardrum. RAG-S The middle ear includes the tympanic cavity and the three ossicles. MARCO what currency BART The currency needed in Scotland is Pound sterling. needed in RAG-T Pound is the currency needed in Scotland. scotland RAG-S The currency needed in Scotland is the pound sterling. BART ?This state has the largest number of counties in the U.S. Jeopardy Washington RAG-T It’s the only U.S. state named for a U.

In [ ]:
Image

['@--- ee ee ee ee ee ee ee ee ee ee eee The middle ear includes End-to-End Backprop through q and pe the tympanic cavity and the three ossicles. (y) Define "middle ear" (x) Question Answering: Question Query Retriever py Document Generator pg “fnower Generation Index. (Non-Parametric) (Parametric) d(z) supports (y) Barack Obama was born in Hawaii. (x) q(x) Fact Verification: Fact Query Fact Verification: Label Generation The Divine This 14th century work Comedy (x) is divided into 3 Jeopardy Question Generation: Answer Query sections: "Inferno", "purgatorio" & "Paradiso" @) Question Generation',
 '| | | |',
 'Bee TT % 80 Porm Sa SRS nana ga g / Z fr = 70 2 | / 3 RAG TORRE 2 nf |! g <= RAG-Tok B-1 Ba Ze H=- RAGSeq RL a ; 3 Zs == RAG-Seq BA Q 50 > 50 ZO — reactor | & 3 soft === RAGSeq | Z 40 2 4s rr rr rr nr) rr nr K Retrieved Docs K Retrieved Docs K Retrieved Docs',
 'View full instructions Which sentence is more factually true? View tool guide Select an option Subject : Hemingway eI N

# Model loading

## Chat Model

In [ ]:
llm = HuggingFaceEndpoint(
    repo_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    # repo_id = "perplexity-ai/r1-1776",
    task="text-generation"
  )
# llm=HuggingFaceEndpoint(repo_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0",task="text-generation")
hf_model = ChatHuggingFace(llm=llm)

try:
  hf_model.invoke("Hi I am Lokesh")
except Exception as e:
  print(e)

In [ ]:
gemini_model = ChatGoogleGenerativeAI(model='gemini-1.5-flash')
gemini_model.invoke("Hi I am Lokesh")

AIMessage(content='Hello Lokesh!  How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run--37a5e30c-8d2f-4cdc-8552-d43bf1687762-0', usage_metadata={'input_tokens': 5, 'output_tokens': 13, 'total_tokens': 18, 'input_token_details': {'cache_read': 0}})

In [ ]:
groq_model = ChatGroq(model="deepseek-r1-distill-llama-70b")
groq_model.invoke("Hi I am Lokesh")

AIMessage(content='<think>\n\n</think>\n\nHi Lokesh! How can I assist you today? 😊', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 8, 'total_tokens': 26, 'completion_time': 0.100251449, 'prompt_time': 0.000162046, 'queue_time': 0.206830667, 'total_time': 0.100413495}, 'model_name': 'deepseek-r1-distill-llama-70b', 'system_fingerprint': 'fp_1bbe7845ec', 'finish_reason': 'stop', 'logprobs': None}, id='run--2f4b3146-8041-4aa7-804b-01ec6ab0e064-0', usage_metadata={'input_tokens': 8, 'output_tokens': 18, 'total_tokens': 26})

In [ ]:
open_ai_model = ChatOpenAI(model="gpt-4o")
open_ai_model.invoke("Hi I am Sunny")

AIMessage(content='Hello Sunny! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 11, 'total_tokens': 22, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_55d88aaf2f', 'id': 'chatcmpl-BX7mPDFSz18yDIGLmS3V2f3Zo5IbS', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--f8fba0e1-e6f3-4bd6-ba18-05afe1209319-0', usage_metadata={'input_tokens': 11, 'output_tokens': 11, 'total_tokens': 22, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
claude_model = ChatAnthropic(model="claude-2")
claude_model.invoke("Hi I am Sunny")

AIMessage(content='Hello Sunny, nice to meet you!', additional_kwargs={}, response_metadata={'id': 'msg_016uaA299mfgGchcZWYUk5NJ', 'model': 'claude-2', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 14, 'output_tokens': 13, 'server_tool_use': None}, 'model_name': 'claude-2'}, id='run--6d07ddd4-3aec-469f-9477-a3d78baa395e-0', usage_metadata={'input_tokens': 14, 'output_tokens': 13, 'total_tokens': 27, 'input_token_details': {'cache_read': 0, 'cache_creation': 0}})

In [ ]:
llm = HuggingFacePipeline.from_model_id(
    model_id='TinyLlama/TinyLlama-1.1B-Chat-v1.0',
    task='text-generation',
    pipeline_kwargs={
        "temperature": 0.5,
        "max_new_tokens":100
        }
)

model = ChatHuggingFace(llm=llm)
model.invoke("Hi I am Lokesh")

##Embeding Model

In [ ]:
# @markdown # OpenAI

embedding_model = "text-embedding-3-large" # @param ["text-embedding-3-large","text-embedding-3-small","text-embedding-ada-002"]
dimensions = 64 #@param {type:"integer"}
set_dimension = True # @param {type:"boolean"}
query = "India is a growing country" # @param {"type":"string","placeholder":"India is a growing country"}

if set_dimension:
  openai_embedding = OpenAIEmbeddings(
      model=embedding_model,
      dimensions=dimensions,
  )
else:
  openai_embedding = OpenAIEmbeddings(
      model=embedding_model,
  )

result = openai_embedding.embed_query(query)
print(len(result),result)

64 [-0.22190459072589874, 0.2781512439250946, -0.019638171419501305, 0.047116201370954514, 0.007093434687703848, 0.0018865261226892471, -0.15810702741146088, 0.09076514095067978, 0.03236108645796776, -0.12227867543697357, 0.08452407270669937, -0.04599897190928459, 0.11403430253267288, -0.10378662496805191, -0.10417187958955765, 0.014938108623027802, 0.08899299055337906, -0.20634044706821442, -0.0308393444865942, -0.1088719367980957, -0.19863542914390564, -0.11588350683450699, 0.045228470116853714, -0.17860238254070282, -0.21543237566947937, 0.09045694023370743, -0.036541059613227844, 0.003484114073216915, -0.15533322095870972, 0.13183289766311646, 0.09477175027132034, 0.014052031561732292, 0.06156311556696892, -0.05451301857829094, -0.03172542154788971, 0.27722662687301636, -0.024501964449882507, -0.054859746247529984, -0.07520099729299545, -0.08383062481880188, -0.007314953953027725, 0.015670085325837135, -0.11287855356931686, 0.4059004783630371, 0.1887730062007904, 0.0106040341779589

In [ ]:
# @markdown # Google GenAi

embedding_model = "models/gemini-embedding-exp-03-07" # @param ["models/gemini-embedding-exp-03-07","models/text-embedding-004","models/embedding-001"]
task_type = "retrieval_query" # @param ["None","task_type_unspecified","retrieval_query","retrieval_document","semantic_similarity","classification","clustering"]
transport = "None" # @param ["None","rest","grpc","grpc_asyncio"]
query = "India is a growing country" # @param {"type":"string","placeholder":"India is a growing country"}

func = lambda x : None if x=="None" else x
task_type = func(task_type)
transport = func(transport)

google_embedding = GoogleGenerativeAIEmbeddings(
    model=embedding_model,
    task_type=task_type,
    transport=transport
)

result = google_embedding.embed_query(query)
print(len(result),result)


3072 [-0.01328858733177185, -0.008076989091932774, -0.02803654596209526, -0.0385795421898365, -0.0012177267344668508, -0.009753282181918621, 0.018366066738963127, 0.0390302836894989, 0.005043548997491598, -0.023734668269753456, -0.011821064166724682, 0.006893169600516558, 0.019229836761951447, 0.03314683958888054, 0.11351391673088074, 0.020717905834317207, 0.0073205651715397835, -0.019075468182563782, 0.006459957454353571, -0.0205469261854887, -0.003892261302098632, 0.01249883696436882, 0.01834138110280037, 0.004717701114714146, -0.005373682361096144, -0.0032776377629488707, 0.006898732855916023, 0.02193414606153965, 0.021265285089612007, 0.014482101425528526, 0.005145769566297531, -0.027150483801960945, 0.023643454536795616, 0.012725071050226688, 0.017535502091050148, 0.015545469708740711, 0.022201204672455788, 0.0026000298094004393, 0.02127012610435486, 0.005257639102637768, -0.02616986259818077, 0.009095565415918827, -0.022166412323713303, -0.007171641103923321, -0.02820508368313312

In [ ]:
# @markdown # Hugging Face
model_name = "all-MiniLM-L6-v2" # @param ["BAAI/bge-en-icl","all-MiniLM-L6-v2"]
query = "India is a growing country" # @param {"type":"string","placeholder":"India is a growing country"}
huggingface_embeddings=HuggingFaceEmbeddings(model_name=model_name)

result = huggingface_embeddings.embed_query(query)
print(len(result),result)

# Prepare for VB

## Summary for the Text

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
prompt_text="""You are an assistant tasked with summarizing text for retrieval. \
    These summaries will be embedded and used to retrieve the raw text elements. \
    Give a concise summary of the table or text that is well optimized for retrieval.text: {element} """

prompt_template = ChatPromptTemplate.from_template(prompt_text)

summarize_chain = {"element": lambda x: x} | prompt_template | open_ai_model | StrOutputParser()

summarize_chain.get_graph().print_ascii()

+------------------------+ 
| Parallel<element>Input | 
+------------------------+ 
             *             
             *             
             *             
        +--------+         
        | Lambda |         
        +--------+         
             *             
             *             
             *             
  +--------------------+   
  | ChatPromptTemplate |   
  +--------------------+   
             *             
             *             
             *             
      +------------+       
      | ChatOpenAI |       
      +------------+       
             *             
             *             
             *             
    +-----------------+    
    | StrOutputParser |    
    +-----------------+    
             *             
             *             
             *             
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  


In [ ]:
text_summary=summarize_chain.batch(NarrativeText,{"max_concurrency": 5})
text_summary

['Authors: Patrick Lewis and Ethan Perez',
 'Authors: Aleksandra Piktus, Fabio Petroni, Vladimir Karpukhin, Naman Goyal, Heinrich Küttler.',
 'This text lists individuals associated with a project or paper: Mike Lewis, Wen-tau Yih, Tim Rocktäschel, Sebastian Riedel, and Douwe Kiela. The presence of symbols † and ‡ suggests affiliations or contributions are indicated but not detailed in the text.',
 'This text lists affiliations: Racebook AI Research, University College London, and New York University.',
 'Email address: plewis@fb.com',
 'This text examines retrieval-augmented generation (RAG) models that combine pre-trained parametric memory with non-parametric memory for language tasks. Unlike traditional language models, these RAG models incorporate a seq2seq framework linked to a dense vector index of Wikipedia, accessed via a pre-trained neural retriever. Two RAG approaches are compared: one using consistent retrieved passages for the entire sequence and another using varied passag

In [ ]:
text_summary[3]

In [ ]:
import re

def clean_text(text:str):
  return re.sub(r"<think>.*?</think>\s*","",text,flags=re.DOTALL)


## Summary for Table

In [ ]:
prompt_text = """You are an AI Assistant tasked with summarizing tables for retrieval. \
    These summaries will be embedded and used to retrieve the raw table elements. \
    Give a concise summary of the table that is well optimized for retrieval. Table:{element} """

prompt = ChatPromptTemplate.from_template(prompt_text)
summarize_chain = {"element": lambda x: x} | prompt | open_ai_model | StrOutputParser()


summarize_chain.get_graph().print_ascii()

+------------------------+ 
| Parallel<element>Input | 
+------------------------+ 
             *             
             *             
             *             
        +--------+         
        | Lambda |         
        +--------+         
             *             
             *             
             *             
  +--------------------+   
  | ChatPromptTemplate |   
  +--------------------+   
             *             
             *             
             *             
      +------------+       
      | ChatOpenAI |       
      +------------+       
             *             
             *             
             *             
    +-----------------+    
    | StrOutputParser |    
    +-----------------+    
             *             
             *             
             *             
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  


In [ ]:
table_summaries = summarize_chain.batch(Table, {"max_concurrency": 5})
table_summaries

['This table presents a comparison of various models on their performance metrics, including B-1 (BLEU-1), QB-1, R-L (ROUGE-L), and Label Accuracy. Specific models featured are T5-11B, T5-11B+SSM, Open Realm, DPR, BART, RAG-Token, and RAG-Sequence, each showing diverse results. The table highlights Closed Book settings, with metrics like 52, 34.5, and accuracy figures like 40.4 and 44.1 for specific models, alongside Open REALM and Book performance measures such as 49.8*, 49.9*, and ranges up to 89.5. Additionally, there are references to state-of-the-art (SotA) benchmarks. The data is displayed in a structured format to enable comparison of model capabilities under various task configurations.',
 'The table contrasts multiple AI model generations for various tasks and inputs. For defining the "middle ear," BART provides an incorrect looped definition, RAG-T describes it accurately as being internal to the eardrum, and RAG-S specifies it includes the tympanic cavity and ossicles. Regar

## Summary for Images

In [ ]:
import base64, os
from langchain_core.messages import AIMessage, HumanMessage

def encode_image(image_path):
  with(open(image_path,"rb")) as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

def image_summarize(imag_base64,prompt):

  msg = gemini_model.invoke(
      [
          HumanMessage(
              content=[
                  {
                      "type":"text",
                      "text":prompt
                  },
                  {
                      "type":"image_url",
                      "image_url":{
                          "url": f"data:image/jpeg;base64,{imag_base64}"
                      }
                  }
              ]
          )
      ]
  )

  return msg.content

def generate_imag_summaries(path):

  image_base64_list = []

  image_summarises = []

  prompt = """You are an assistant tasked with summarizing images for retrieval. \
    These summaries will be embedded and used to retrieve the raw image. \
    Give a concise summary of the image that is well optimized for retrieval."""


  for img_file in sorted(os.listdir(path)):
    if img_file.endswith(".jpg") or img_file.endswith(".jpeg") or img_file.endswith(".png"):
      image_path = os.path.join(path, img_file)
      image_base64 = encode_image(image_path)
      image_base64_list.append(image_base64)
      image_summaries = image_summarize(image_base64,prompt)
      image_summarises.append(image_summaries)

  return image_base64_list, image_summarises

In [ ]:

image_dir_path = "extracted_data"
image_base64_list, image_summarises = generate_imag_summaries(image_dir_path)

image_summarises

['Hemingway authorship quiz:  "The Sun Also Rises" and birthplace fact check.  Multiple choice question.',
 'Diagram of end-to-end backpropagation through a query and a generator showing a query encoder, retriever, document index, and generator with examples of question answering, fact verification, and question generation.',
 'light-blue background with two dark-blue vertical bars',
 'Three graphs showing NQ Exact Match, NQ Answer Recall@K, and Bleu-1/Rouge-L score for RAG-Tok and RAG-Seq models against Fixed DPR and BM25 baselines, plotted against the number of retrieved documents (K).',
 'Dataset sizes for Natural Questions, TriviaQA, WebQuestions, CuratedTrec, Jeopardy Question Generation, MS-MARCO, FEVER-3-way, and FEVER-2-way question answering tasks.  Train, development, and test set sizes are provided.',
 'Table comparing question answering model performance across multiple datasets (NQ, TQA, WQ, Jeopardy, MSMARCO, FVR3, FVR2).  Results include accuracy scores for different mod

# Store data in vectordb

In [ ]:
from langchain_core.documents import Document
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain.retrievers.multi_vector import MultiVectorRetriever
import uuid

In [ ]:
os.environ["GROQ_API_KEY"] = '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'


In [ ]:
def create_multi_vector_retriver(
    vectorstore,
    text_summaries ,texts,
    table_summaries,tables,
    image_summarises, images
):

  retriever=MultiVectorRetriever(
      vectorstore=vectorstore,
      docstore=InMemoryStore(),
      id_key="doc_id",
  )

  def add_docs(retriever,doc_summaries,doc_contents):

    doc_ids = [ str(uuid.uuid4()) for _ in range(len(doc_summaries))]

    summary_docs = [
        Document(
            page_content=doc_summary,
            metadata={"doc_id":doc_id}
        )
        for doc_summary,doc_id in zip(doc_summaries,doc_ids)
    ]

    retriever.vectorstore.add_documents(summary_docs)
    retriever.docstore.mset(
        list(zip(doc_ids,doc_contents))
    )

  add_docs(retriever,text_summaries,NarrativeText)
  add_docs(retriever,table_summaries,Table)
  add_docs(retriever,image_summarises,image_summarises)

  return retriever

# embedding_model=load_model("embedding")

vectorstore=Chroma(collection_name="MMRAG",embedding_function=huggingface_embeddings)
retriever_multi_vector=create_multi_vector_retriver(
    vectorstore,
    text_summary,NarrativeText,
    table_summaries,Table,
    image_summarises, image_base64_list
)



<ipython-input-32-4db8d4c6874a>:39: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore=Chroma(collection_name="MMRAG",embedding_function=huggingface_embeddings)


In [ ]:
retriever_multi_vector

MultiVectorRetriever(vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x79126e41d4d0>, docstore=<langchain_core.stores.InMemoryStore object at 0x79126e14f8d0>, search_kwargs={})

In [ ]:
retriever_multi_vector.get_relevant_documents(
    "Why We combine a pre-trained retriever (Query Encoder + Document Index) with a pre-trained seq2seq model (Generator) and fine-tune end-to-end?"
)

<ipython-input-34-1b27466b39d0>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever_multi_vector.get_relevant_documents(


['We endow pre-trained, parametric-memory generation models with a non-parametric memory through a general-purpose ﬁne-tuning approach which we refer to as retrieval-augmented generation (RAG). We build RAG models where the parametric memory is a pre-trained seq2seq transformer, and the non-parametric memory is a dense vector index of Wikipedia, accessed with a pre-trained neural retriever. We combine these components in a probabilistic model trained end-to-end (Fig. 1). The retriever (Dense Passage Retriever [26], henceforth DPR) provides latent documents conditioned on the input, and the seq2seq model (BART [32]) then conditions on these latent documents together with the input to generate the output. We marginalize the latent documents with a top-K approximation, either on a per-output basis (assuming the same document is responsible for all tokens) or a per-token basis (where different documents are responsible for different tokens). Like T5 [51] or BART, RAG can be ﬁne-tuned on an

In [ ]:
retriever_multi_vector.invoke(
    "Why We combine a pre-trained retriever (Query Encoder + Document Index) with a pre-trained seq2seq model (Generator) and fine-tune end-to-end?"
)

['We endow pre-trained, parametric-memory generation models with a non-parametric memory through a general-purpose ﬁne-tuning approach which we refer to as retrieval-augmented generation (RAG). We build RAG models where the parametric memory is a pre-trained seq2seq transformer, and the non-parametric memory is a dense vector index of Wikipedia, accessed with a pre-trained neural retriever. We combine these components in a probabilistic model trained end-to-end (Fig. 1). The retriever (Dense Passage Retriever [26], henceforth DPR) provides latent documents conditioned on the input, and the seq2seq model (BART [32]) then conditions on these latent documents together with the input to generate the output. We marginalize the latent documents with a top-K approximation, either on a per-output basis (assuming the same document is responsible for all tokens) or a per-token basis (where different documents are responsible for different tokens). Like T5 [51] or BART, RAG can be ﬁne-tuned on an

# Image data processing

In [ ]:
import io, re
from IPython.display import HTML, display
from PIL import Image

In [ ]:
def is_image_data(b64data)->bool:
  image_signatures = {
      b"\xFF\xD8\xFF": "jpg",
      b"\x89\x50\x4E\x47\x0D\x0A\x1A\x0A": "png",
      b"\x47\x49\x46\x38": "gif",
      b"\x52\x49\x46\x46": "webp",
  }

  try:
    header = base64.b64decode(b64data)[:8] # Decode and get first 8 bytes
    for sig, format in image_signatures.items():
      if header.startswith(sig):
        return True
    return False
  except:
    return False

def looks_like_base64(sb):
  return re.match("[A-za-z0-9+/]+[=]{0,2}$",sb) is not None

def resize_base64_image(base64_string,size=(128,128)):
  img_data = base64.b64decode(base64_string)
  img = Image.open(io.BytesIO(img_data))
  resized_img = img.resize(size,Image.LANCZOS)

  buffered = io.BytesIO()
  resized_img.save(buffered, format=img.format)
  return base64.b64encode(buffered.getvalue()).decode('utf-8')

def split_image_text_types(docs):
  """
  Split base64-encoded images and text
  """

  b64_images = []
  texts = []

  for doc in docs:

    if isinstance(doc, Document):
      doc = doc.page_content
    if looks_like_base64(doc) and is_image_data(doc):
      doc =resize_base64_image(doc,size=(1300,600))
      b64_images.append(doc)
    else:
      texts.append(doc)

  return {
      "images":b64_images,
      "texts":texts
  }

def img_prompt_func(data_dict):
  """
  Join the context into a single string
  """

  print(data_dict)
  formatted_text = "\n".join(data_dict["context"]["texts"])

  messages = []

  if data_dict["context"]["images"]:
    for img in data_dict["context"]["images"]:
      messages.append(
          {
              "type":"image_url",
              "image_url":{
                  "url": f"data:image/jpeg;base64,{img}"
              }
          }
      )
  text_message = {
      "type":"text",
      "text": (
          "You are a helpful assistant.\n"
          "You will be given a mixed info(s) .\n"
          "Use this information to provide relevant information to the user question. \n"
          f"User-provided question: {data_dict['question']}\n\n"
          "Text and / or tables:\n"
          f"{formatted_text}"
      ),
  }

  messages.append(text_message)

  return [HumanMessage(content=messages)]



In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

def muti_model_rag_chain(retriever):
  chain = (
      {
          "context":retriever | RunnableLambda(split_image_text_types),
          "question":RunnablePassthrough(),

      }
      | RunnableLambda(img_prompt_func)
      | open_ai_model
      | StrOutputParser()
  )

  return chain

chain_mm_rag = muti_model_rag_chain(retriever_multi_vector)

chain_mm_rag.get_graph().print_ascii()

              +---------------------------------+           
              | Parallel<context,question>Input |           
              +---------------------------------+           
                     ****               ****                
                  ***                       ***             
                **                             ***          
 +----------------------+                         **        
 | MultiVectorRetriever |                          *        
 +----------------------+                          *        
             *                                     *        
             *                                     *        
             *                                     *        
+------------------------+                  +-------------+ 
| split_image_text_types |                  | Passthrough | 
+------------------------+                  +-------------+ 
                     ****               ****                
                        

In [ ]:
chain_mm_rag.invoke(
    """Explain any images / figures in the paper with Left: NQ performance as more documents are retrieved. Center: Retrieval recall performance\
in NQ. Right: MS-MARCO Bleu-1 and Rouge-L as more documents are retrieved.
  """
)

{'context': {'images': [], 'texts': ['Three graphs showing NQ Exact Match, NQ Answer Recall@K, and Bleu-1/Rouge-L score for RAG-Tok and RAG-Seq models against Fixed DPR and BM25 baselines, plotted against the number of retrieved documents (K).', 'to more effective marginalization over documents. Furthermore, RAG can generate correct answers even when the correct answer is not in any retrieved document, achieving 11.8% accuracy in such cases for NQ, where an extractive model would score 0%.', 'Effect of Retrieving more documents Models are trained with either 5 or 10 retrieved latent documents, and we do not observe signiﬁcant differences in performance between them. We have the ﬂexibility to adjust the number of retrieved documents at test time, which can affect performance and runtime. Figure 3 (left) shows that retrieving more documents at test time monotonically improves Open-domain QA results for RAG-Sequence, but performance peaks for RAG-Token at 10 retrieved documents. Figure 3 

'In the provided text, the references to "Figure 3" likely correspond to three separate graphs or charts each demonstrating different performance metrics as a function of the number of documents retrieved by various models.\n\n1. **Left: NQ (Natural Questions) Performance as More Documents are Retrieved**: \n   - This graph shows the Exact Match score for the NQ dataset as more documents are retrieved. The model comparison includes RAG-Sequence and RAG-Token against the Fixed DPR and BM25 baselines.\n   - The explanation highlights that as more documents are retrieved, there is a monotonic improvement in Open-domain QA results for RAG-Sequence, meaning its performance consistently improves.\n   - For RAG-Token, the performance peaks when 10 documents are retrieved, suggesting an optimal point beyond which additional documents do not significantly contribute to performance improvements.\n\n2. **Center: Retrieval Recall Performance in NQ**:\n   - In this context, "Retrieval Recall@K" lik